**PROJECT 2 - VISUALIZATION INFORMATION**

AUTHORS:

    ALINA CASTELL BLASCO    
    GERARD MARTIN PEY   
    JOSE ÀNGEL MOLA AUDÍ

In [1]:
!pip install geopandas
!pip install folium

In [1]:
import pandas as pd
import altair as alt
import folium  # used for plotting the map of Barcelona
import pprint
import geopandas as gpd

**Preprocessing:**

- First of all we selected the 10 departments we were going to study. A little investigation was needed in those to ensure the charts will be rich enough. Those meant ensuring that the locations were diverse enough and that we compared departments having different methodologies when it comes to publications.

- After chosing the departments, we searched more information such as the the number of employees grouped by categories, the school, the location of their section that could enrich more the visualization,etc.

- Later we did an inner join of these departments with the initial data set in order to obtain only information from the requested departments.

- Finally, we put the Research Group acronyms as a column of the DataFrame.

**Questions to interactively answer:**

1) Visual comparison of the number of publications along time of two departments interactively selected.

2) Interactive map of Catalonia/Barcelona with points of the locations of departments.

3) Interactive visualization for the stats of a department.

4) Interactive chart of the publications vs department size.

5) Visual comparison of the number of publications along time with a slider on average publications per year.

6) Interactive chart of the intersection of the selected 10 departments with the 40 areas of research with more production.

In [2]:
alt.data_transformers.disable_max_rows()
data = pd.read_csv("DATA-RESEARCH_DEFINITIU.csv", delimiter = ";")
selection = ["TSC", "FIS", "PA", "MAT", "CS", "EM", "OO", "DEGE", "DCMEM", "DECA"]
departments = pd.DataFrame(selection, columns = ["DEPARTMENTS"])

In [3]:
data_proj2 = pd.merge(data,departments,how='inner',left_on=['DEPT ACRONIMS'],right_on=['DEPARTMENTS'])
data_proj2 = data_proj2.drop(["DEPARTMENTS"], axis = 1)
l = data["GRUP DE RECERCA UPC"].unique()
acronims_grups = {}
for elem in l:
    if type(elem) == str:
        acronims_grups[elem] = elem.split()[0]
acronym = pd.DataFrame(acronims_grups.items(), columns = ["Name", "GRUP DE RECERCA UPC ACRONIM"])
data_proj2 = pd.merge(data_proj2,acronym,how='inner',left_on=['GRUP DE RECERCA UPC'],right_on=['Name'])
data_proj2.drop(["Name"], axis = 1)
data_proj2 = data_proj2[["DATA PUBLICACIO", "ID_ARTICLE", "ISSN REVISTA", "GRUPS ÀMBITS", "ÀMBIT CONEIXEMENT DE LA REVISTA", "ID_AUTOR", "DEPT ACRONIMS", "GRUP DE RECERCA UPC"]]

In [4]:
data1 = data_proj2
data1["FULL DATE"] = pd.DatetimeIndex(data1['DATA PUBLICACIO']).day.map(str) + '-' + pd.DatetimeIndex(data1['DATA PUBLICACIO']).month.map(str) + '-' + pd.DatetimeIndex(data1['DATA PUBLICACIO']).year.map(str)
data1 = data1.drop_duplicates(subset = ["ID_ARTICLE"])

Also, we define the table with information obtained from UPC sources about the members of each of the 10 selected departments.

In [ ]:
# Dataset with info about members of the departments
table = pd.read_csv("tauladept.csv", delimiter = ";", decimal=",")
table

**QUESTION 1:**

Firstly we had a very different idea than what we finally got. We wanted to take advantage of the fact that the departments were greatly reduced and for this reason, we have used a line chart that shows the progression of each department for each year. With the interactivity we have been able to compare each department well with another and see their behavior with respect to the average.

To have the average of the departments per year, we had to create this dataframe:

In [ ]:
data1["Year"] = pd.DatetimeIndex(data1['DATA PUBLICACIO']).year.map(int)
data_year_mean = (data1.groupby(['Year'])['Year'].count()/len(departments)).reset_index(name='Mean')

However, we had an initial doubt about what the aggregation level of the chart would be. We've checked and have a total of 2874 different dates where we initially saw it as a good idea to consider days instead of years so we could get a deeper breakdown of everything:

In [ ]:
data_list = data1["FULL DATE"].unique()
print(data_list)
print("We have %d different dates" % len(data_list))

But, we were able to see how this was not a good idea:

In [ ]:
multi_sel = alt.selection_multi(fields = ["DEPT ACRONIMS"], bind = "legend", name = "DEPTS")

line_agg = alt.Chart(data1).mark_line(size = 2).encode(
    x=alt.X('FULL DATE:T', title = "Years"),
    y=alt.Y('count(DEPT ACRONIMS):Q', title = "Number of articles", scale=alt.Scale(domain=(0, 100))),
    color=alt.condition(multi_sel, alt.Color('DEPT ACRONIMS:N', legend = alt.Legend(columns=1), title = "Departments", scale = alt.Scale(scheme="category20")), alt.value("transparent"))
).properties(width=800,height=400).properties(title = 'Production of departments among all years')


mean = alt.Chart(data_year_mean).mark_line(size = 4,color = 'yellow').encode(
    y = alt.Y('Mean:Q'),
    x = alt.X('FULL DATE:N'),
    opacity = alt.value(0.5)
)

line_aux_agg = alt.Chart(data1).mark_line(size = 2).encode(
    x=alt.X('FULL DATE:T', title = "Years"),
    y=alt.Y('count(DEPT ACRONIMS):Q', title = "Number of articles"),
    color=alt.condition(multi_sel, alt.Color('DEPT ACRONIMS:N', legend = alt.Legend(columns=1), title = "Departments", scale = alt.Scale(range=['transparent'])), alt.value("lightgray"))
).properties(width=800,height=400)

exercise_1_agg = alt.layer(line_agg+line_aux_agg+line_agg).interactive().add_selection(multi_sel)
exercise_1_agg

As we can see, we have peaks at the beginning of each year since many documents are dated incorrectly and put as generic the value of the first day of the year, in addition we can appreciate a large overlap.

So we add the year to every observation:

Therefore, we proceed to treat the data as annual, that is, our variables are as follows:

- x to know the year
- and to know the number of documents from each department
- color to know which department we are dealing with

It should be noted that when a selection is made, the rest of the selected elements become gray, so that our vision focuses only on the selected one, but we can also see what the behavior of the selected department is with respect to the rest. There is a line of an opaque yellow color that always remains, this is the average of the departments.


Now we will discuss how we made the plot so that it comes out as we said. We added two plots:

- One when we select departments, the lines of the selected ones appear with the corresponding color and those of the unselected ones disappear.
- One when we select departments, those that are selected disappear and those that are not selected become gray.

This sum produces the result we want, but we had to deal with the fact that sometimes the unselected ones overlapped the selected ones and you could see the gray lines on top of the colored ones. We have solved this by adding the first graph at the end, so we make sure that the color ones are on top, which is what we are interested in.


The selected palette has been one that does not contain yellow or gray, in order to see everything more clearly.

In [ ]:
multi_sel = alt.selection_multi(fields = ["DEPT ACRONIMS"], bind = "legend", name = "DEPTS")

line = alt.Chart(data1).mark_line(size = 2).encode(
    x=alt.X('Year:N', title = "Years", axis = alt.Axis(labelAngle=-45)),
    y=alt.Y('count(DEPT ACRONIMS):Q', title = "Number of articles"),
    color=alt.condition(multi_sel, alt.Color('DEPT ACRONIMS:N', legend = alt.Legend(columns=1), title = "Departments", scale = alt.Scale(scheme="category20")), alt.value("transparent"))
).properties(width=800,height=400).properties(title = 'Production of departments among all years')

mean = alt.Chart(data_year_mean).mark_line(size = 4,color = 'yellow').encode(
    y = alt.Y('Mean:Q'),
    x = alt.X('Year:N'),
    opacity = alt.value(0.5)
)

line_aux = alt.Chart(data1).mark_line(size = 2).encode(
    x=alt.X('Year:N', title = "Years"),
    y=alt.Y('count(DEPT ACRONIMS):Q', title = "Number of articles"),
    color=alt.condition(multi_sel, alt.Color('DEPT ACRONIMS:N', legend = alt.Legend(columns=1), title = "Departments", scale = alt.Scale(range=['transparent'])), alt.value("lightgray"))
).properties(width=800,height=400)

exercise_1 = alt.layer(line+line_aux+line+mean).interactive().add_selection(multi_sel)
exercise_1

**QUESTION 2:**

- First of all we decided to pick light grey as a background color for the map because is a soft color that helps to highlight other colors placed on it.
- After taht, we selected the adresses provided by Futur of each of the ten departments chosen. However, most of them were located too near, leading to clutter and oclusion in the map. 
- Therefore, we decided to restrict the map to only the regions of Catalonia that had departments located in them, in order to make a better use of space and be able to zoom in the area of interest.
- Due to the closeness of most departments, clutter was still an issue. We opted then for taking into account adresses of department sections. By doing that we were able to enrich the visualization through adding new locations to the map.
- At one point we intended to show all the sections of departments in the map and place lines between sections of the same department to see in how many locations each of them is present. Nevertheless, by only trying that with two departments, clutter lead to poor visualization. Consequently, we decided to discard this option.
- Some clutter was still visible. However, bearing in mind that UPC schools are not distributed between more than ten regions among Catalonia, this is inevitable. We considered therefore this one to be a proper distribution for the points. However, there was oclusion between two departments; we solved that by changing a little bit the coordinates of one of them, making cliking and hovering available for both.
- When it comes to marks and in order to add visually attractive variables, we decided to substitute the typical points for emojis that represented the discipline related to each department. Therefore, as the legend only accepts colors keeping for this chart has no sense as it is not providing information, so we decided to erase it.

In [10]:
# Redefine the previously defined table with information about department members
df2 = table

# Grouping dataset by departments
df1_grouped = data_proj2.groupby("DEPT ACRONIMS", sort=True, as_index=False).nunique()

# Ensuring that the key has the same spelling in both datasets
df2 = df2.rename(columns={"DeptAcronym": "DEPT ACRONIMS"})
df1_grouped = df1_grouped.rename(columns={"DEPT ACRONIMS": "DEPT ACRONIMS"})

# Merging using departments as keys
df = pd.merge(df1_grouped,df2, how="inner", on="DEPT ACRONIMS")
df = df.rename(columns={"ID_ARTICLE":"Number of publications", "ISSN REVISTA":"Magazines", "ÀMBIT CONEIXEMENT DE LA REVISTA":"Research fields", "ID_AUTOR":"Number of authors","GRUP DE RECERCA UPC": "Research groups"})

# Adding emojis to use as markers
df['emoji'] = [{'CS':'💻', 'DCMEM':'🧱', 'DECA':'🛠️', 'DEGE':'🎨', 'EM':'🔧', 'FIS':'👩‍🔬', 'MAT':'➕', 'OO':'👓', 'PA':'🏙️', 'TSC':'📱'}[dept] for dept in df['DEPT ACRONIMS']]

In [ ]:
# Read the map of Catalonia
choro_data = gpd.read_file('comarques-compressed.geojson')
choro_data = choro_data.loc[choro_data['nom_comar'].isin(['Barcelonès','Vallès Occidental','Garraf','Baix Llobregat', 'Bages'])]

# Create a selection
xSel = alt.selection_single(encodings = ['color'], fields = ["DEPT ACRONIMS"],empty = 'none')

background = alt.Chart(choro_data).mark_geoshape(fill = 'lightgray', stroke = 'white').properties(width = 300, height = 500)
points = alt.Chart(df).mark_text(filled=True).encode( #,tooltip={'content':'data'}
    longitude = 'Longitude:Q',
    latitude = 'Latitude:Q',
    size = alt.SizeValue(15),
    text='emoji',
    color = alt.condition(xSel, alt.Color('DEPT ACRONIMS:N', legend=None), alt.value('black')),
    tooltip = ['DeptName:N', 'Number of authors:Q', 'Number of publications:Q' ,'Total:Q', 'Faculty:Q', 'Associate:Q', 'Beca:Q', 'Other:Q', 'Location:N', 'School:N']
).add_selection(
    xSel
).properties(
    title = 'Location of UPC Departments'
).properties(width = 300, height = 500)

exercise_2 = background+points
exercise_2

**QUESTION 3:**

- The first idea was to represent some variables about each department using a line chart to show their evolution among years. However, this design was too simple and already used in the vis.
- We decided then to switch to a different design that used a slider for years. This way we were able to show more clearly how was data for each department at each month of a given year.
- Additionally, we opted also for using bars instead of lines. As we wanted to show more variables we considered to be a good idea to use a grouped bar chart, showing diferent data per month.
- We added also a selection into the legend to provide the user with the power of selecting which stats are more interesting for him/her to visualize. Initially all four variables are selected and the user can select a subset of them by clicking to them in the legend of the grouped bar chart (remeber that for the first selection is enough to click but for the rest of them ).
- As there is no data provided for certain months and departments (months where that department had no publications), the chart initially did not consider those months to exist, consequently removing the columns representing them. This was not an issue initially as we considered that not showing empty months was a useful method to use space efficiently. However, when it comes to the final visualization, having a chart of changing size is a trouble. Due to this fact, we decided to make the number of columns fixed for every year, stat and department (always 12 columns; 1 per month) even if some of them were 0. We had to modify the dataframe adding rows with stat value 0 to do so.
- Zoom is provided into the chart to be able to appreciate better the low stats. Slicing up zooms out whereas slicing down zooms in. To move up or down in the chart one has to click and move the slice in the direction he/she wants to go.
- Following the statment, the stats showed are the ones from the selected department in the map.
- Lastly, the chart vanished when the selection was empty, as, for the same reason explained before, when there are no entries, the empty columns are erased, leaving the chart without any of them. In order to solve that, we tried to add the means of the ten departments for each month in each stat.
- We have not been able to get the average lines to appear, despite the code seems appropiate (the cause remains unknown). Nevertheless, layering the averages chart with the grouped bar chart results in solving the size problem, showing all empty columns when the selection is empty. Therefore we have decided to leave that part of code. 

In [ ]:
df3 = data_proj2[["DEPT ACRONIMS","ID_ARTICLE", "ISSN REVISTA", "ÀMBIT CONEIXEMENT DE LA REVISTA", "ID_AUTOR", "GRUP DE RECERCA UPC"]]
# Extract the year and the month of the publication date
df3["Year"] = data_proj2["DATA PUBLICACIO"].str.split(r'\D').str.get(2)
df3["Month"] = data_proj2["DATA PUBLICACIO"].str.split(r'\D').str.get(1)
# Groupby department,year and month
df3_grouped = df3.groupby(["DEPT ACRONIMS", "Month", "Year"], sort=True, as_index=False).nunique()
df3_grouped = df3_grouped.rename(columns={"ID_ARTICLE":"Publications", "ISSN REVISTA":"Magazines", "ÀMBIT CONEIXEMENT DE LA REVISTA":"Research fields", "ID_AUTOR":"Authors","GRUP DE RECERCA UPC": "Research groups"})
# Typos
df3_grouped["Year"] = pd.to_numeric(df3_grouped["Year"])
df3_grouped["Month"] = pd.to_numeric(df3_grouped["Month"])

In [13]:
""" Updates the given parameters according to the order that the gouped dataset follows. In order
to follow that order the first parameter updated is the year, followed by the month and the department.
Parameters:
    d: Department Acronym
    m: Month of the year
    y: Year between 2010 and 2022
"""
def update(d,m,y,i):
    depts = ["CS","DCMEM", "DECA", "DEGE", "EM", "FIS", "MAT","OO","PA", "TSC"]
    y = y+1
    if y>2022:
        y = 2010
        m = m+1
        if m>12:
            m = 1
            if i<len(depts)-1:
                d = depts[i+1]
            i = i+1
    return d,m,y,i
        

In [14]:
# In order to avoid vanishing columns we had to create entries for every combination (department, month, year)
year = 2010
month = 1
dept= "CS"
# Dataframe to store needed non existing combinations of (dept, month, year)
df_aux = pd.DataFrame(columns = ['DEPT ACRONIMS', 'Month','Year', 'Publications', 'Magazines','Research fields','Authors', 'Research groups'])
# Index to traverse department vector in the function 'update'
i = 0
# Traverse the DF
for index,row in df3_grouped.iterrows():
    # Combination not existing in the DF when it is supposed to
    while dept!=row['DEPT ACRONIMS'] or month!=row['Month'] or year!=row['Year']:
        # Add the combination setting the stats to 0
        df_aux.loc[len(df_aux.index)] = [dept, month, year, 0,0,0,0,0]
        dept,month,year,i = update(dept,month, year,i)
    dept,month,year,i = update(dept,month, year,i)
# Put all the columns in the same df
df3_grouped = pd.concat([df3_grouped,df_aux], axis=0)

In [ ]:
df3 = df3[["Month", "Year", "ID_ARTICLE", "ISSN REVISTA", "ÀMBIT CONEIXEMENT DE LA REVISTA",  "ID_AUTOR", "GRUP DE RECERCA UPC"]]
average = df3.groupby(["Month", "Year"], sort=True, as_index=False).nunique()
average = average.rename(columns={"ID_ARTICLE":"Avg_Publications", "ISSN REVISTA":"Avg_Magazines", "ÀMBIT CONEIXEMENT DE LA REVISTA":"Avg_Research fields", "ID_AUTOR":"Avg_Authors","GRUP DE RECERCA UPC": "Avg_Research groups"})
average["Avg_Publications"] = average["Avg_Publications"]/10
average["Avg_Magazines"] = average["Avg_Magazines"]/10
average["Avg_Research fields"] = average["Avg_Research fields"]/10
average["Avg_Authors"] = average["Avg_Authors"]/10
average["Avg_Research groups"] = average["Avg_Research groups"]/10

average["Year"] = pd.to_numeric(average["Year"])
average["Month"] = pd.to_numeric(average["Month"])

average.head(5)

In [ ]:
df3_grouped = pd.merge(df3_grouped,average,how='inner',left_on=['Month', 'Year'],right_on=['Month','Year'])
df3_grouped.head(3)

In [ ]:

# Creation of the slider for years
slider = alt.binding_range(min=int(min(df3_grouped['Year'])), max=int(max(df3_grouped['Year'])), step=1, name = "Year")
# Selection of the year
sel_year = alt.selection_single(name = "Year", fields=['Year'], bind=slider, init={'Year':2022})
# Selection desired statistics
sel_stats = alt.selection_multi(fields=['Stats'],bind='legend')


avg2 = alt.Chart(df3_grouped).mark_rule(color = 'black').encode(
    x = alt.X('Stats:N', axis=None),
    y = alt.Y('Means:Q', axis=alt.Axis(title='')),
).transform_fold(
    as_=['Stats', 'Means'],
    fold=['Avg_publications','Avg_authors', 'Avg_magazines', 'Avg_res']
).transform_filter(
    sel_year
).transform_filter(
    sel_stats
).transform_filter(
    xSel
).properties(
    width=50, height = 500
)

exercise_3 = alt.Chart().mark_bar().encode(
    x = alt.X('Stats:N', axis=None),
    y = alt.Y('Amount:Q', axis=alt.Axis(title='Amount')),
    color = alt.Color('Stats:N', scale=alt.Scale(domain=['Publications','Authors','Magazines','Research fields'])),
).properties(
    width=50, height = 500
).add_selection(
    sel_year
).add_selection(
    sel_stats
).transform_fold(
    as_=['Stats', 'Amount'],
    fold=['Publications','Authors', 'Magazines', 'Research fields']
).transform_filter(
    sel_year
).transform_filter(
    sel_stats
).transform_filter(
    xSel
).interactive()


c = alt.layer(exercise_3,avg2, data=df3_grouped).facet(column=alt.Column('Month', title=None)
).properties(title=alt.TitleParams(text="Relevant stats per month",orient = 'top', anchor = 'middle'))

ex = (exercise_2 | c).properties(
    resolve = alt.Resolve(scale=alt.LegendResolveMap(color=alt.ResolveMode('independent')))
)
ex


**QUESTION 4:** 

In this question we will rebuild de publications vs department size. In order to make it interactive we will make cross selection.

First, what variable do we use for the departments size? In the next table we compare sizes, the one obtained by grouping the number of authors in each department; and the one obtained by UPC sources that states the real number of members in each department.

In [ ]:
# Departments size grouped by Departments and the real size
data4 = data_proj2.groupby('DEPT ACRONIMS')['ID_AUTOR'].nunique().reset_index(name='DEPT_SIZE')
data4['REAL_SIZE'] = table['Total']
data4

We can observe a big difference, the data obtained via UPC is a more reliable source, so for the question we will use the following data. The 'Real Size' obtained from the values of the table and the production of each of the 10 departments from the *csv*.

In [ ]:
data4 = data_proj2.groupby('DEPT ACRONIMS')['ID_ARTICLE'].nunique().reset_index(name='DEPT_PRODUCTION')
data4['REAL_SIZE'] = table['Total']
data4['LATITUDE'] = table['Latitude']
data4['LONGITUDE'] = table['Longitude']
data4

For the question we need a to represent a chart of the previous variables, 'DEPT_PRODUCTION' and 'REAL_SIZE'; and a map of the position of the departments chosen. The map will be used to click on the departments that we want to show on the chart.


The first map that we represented was an interactive map that printed the locations of the departments, nevertheless, this map is plotted with 'folium', where departments cannot be clicked. We show it anyway to observe the difference and that it can be represented in many different ways.

In [ ]:
# plot map
# Create a map object of the city of Barcelona
barcelona_map= folium.Map(location=[41.38879, 2.15899], zoom_start=12)

# Display only accidents where serious injuries where recorded
for lat, lng, label in zip(table.Latitude, table.Longitude, table.DeptName.astype(str)):
    if label!='0':
        folium.features.CircleMarker(
            [lat, lng],
            radius=3,
            color='red',
            fill=True,
            popup=label,
            fill_color='darkred',
            fill_opacity=0.6
        ).add_to(barcelona_map)

# Show map
barcelona_map

Therefore, we create a map that represents the locations of the departments and allowes to click and select them. 

First we represented the whole province of Barcelona, but the pins where very close to each other and they overlapped. So, we zoomed into the map by chosing only the regions where the departments are located, which are Barcelonès and Vallès Occidental. The final result of the map with only those two regions is shown in the complete visualization.

The following code prints the locations of the departments into the previously defined map. These locations are designed to be clicked on, once a department is selected from the map, it is automatically highlighted on the chart next to it. This chart, a scatterplot, shows the number of publications versus the size of each department.

Also, both charts are designed to have the mouse hover over the points of departments. On the map it appears only the names of the departments, while on the chart there is also the exact number of publications and size of the department the mouse is over.

In [ ]:
# Set selection previously defined
xSel
# Condition of the cross selection
color = alt.condition(xSel,alt.value('red'),
                      alt.Color('DEPT ACRONIMS:N',
                                scale=alt.Scale(scheme='category20'), title = "Departments"))

# Use the background defined in the previous question 2

# Interactive chart of the location of departments
points = alt.Chart(data4).transform_aggregate( 
    latitude = 'mean(LATITUDE)', longitude = 'mean(LONGITUDE)',
    groupby = ['DEPT ACRONIMS']
).mark_circle().encode( 
    longitude = 'longitude:Q', latitude = 'latitude:Q',  
    color = color,
    tooltip = ['DEPT ACRONIMS']
).add_selection(xSel)

plot41 = background + points

# Ineractive chart of the department size vs publications
plot42 = alt.Chart(data4).mark_circle().encode(
    x = alt.X('DEPT_PRODUCTION:Q', scale=alt.Scale(domain=[0, 5500], clamp=True), 
              axis=alt.Axis(labelFontSize=8), title = "Production"),
    y = alt.Y('REAL_SIZE:Q', scale=alt.Scale(domain=[0, 270]), title = "Size"),
    opacity = alt.value(0.9),
    color = color,
    tooltip = [
        alt.Tooltip('DEPT ACRONIMS:N', title='Acronim'),
        alt.Tooltip('REAL_SIZE:Q', title='Nº Members'),
        alt.Tooltip('DEPT_PRODUCTION:Q', title='Nº Publications')],
    size = alt.condition(xSel,alt.value(300),alt.value(100))
)

# Combined plot
exercise_4 = plot41 | plot42.properties(width=400,height=500)

plot42 = plot42.properties( title = 'Size vs Production of Departments').properties(width=600, height=500)

exercise_4.properties(
    title = 'Size vs Production of Department selected by Location' 
).configure_title(
    anchor = 'middle'
)

**QUESTION 5**:

We saw this exercise as a kind of visual complement to the first, in which we did it thinking that it would be next to the first in the final visualization. The interactivity allows us to compare the annual production of each department.

As an initial idea, we wanted to make a dot plot, but we know that an indispensable condition of this plot is that it must be ordered, and the fact of having a different order for each year would make us lose the perspective of the order . Therefore, we have decided to make a lollipop chart with the departments ordered alphabetically, and for each year add a yellow horizontal line that corresponds to the average of that year.

Therefore, the variables are:
- x to know the year
- y to know the number of documents from each department
- color to know which department we are dealing with

For every department we have added the number of documents:

In [22]:
input_slider = alt.binding_range(max = 2022, min = 2010, step = 1)

points = alt.Chart(data1).mark_circle(filled = True, size = 60).encode(
    x = alt.X('DEPT ACRONIMS:N',  title = "Departments", axis = alt.Axis(labelAngle=-45)),
    y = alt.Y('count(DEPT ACRONIMS):Q', scale=alt.Scale(domain=(0, 500)),  title = "Number of articles"),
    color = alt.Color('DEPT ACRONIMS:N', scale = alt.Scale(scheme = 'category20'))
).properties(height=400, width = 300).transform_filter(sel_year)

lines = alt.Chart(data1).mark_bar(filled = True, size = 5).encode(
    x = alt.X('DEPT ACRONIMS:N',  title = "Departments", axis = alt.Axis(labelAngle=-45)),
    y = alt.Y('count(DEPT ACRONIMS):Q', scale=alt.Scale(domain=(0, 500)),  title = "Number of articles"),
    color = alt.Color('DEPT ACRONIMS:N', scale = alt.Scale(scheme = 'category20'))
).properties(height=400, width = 300).transform_filter(sel_year).properties(title = 'Production of departments per year')

text = points.mark_text(
    align='center',
    baseline='bottom',
    dy = -4
).encode(
    text='count(DEPT ACRONIMS)'
).transform_filter(sel_year)

mean_y = alt.Chart(data_year_mean).mark_rule(size = 2,color = 'yellow').encode(
    y = alt.Y('Mean:Q'),
    opacity = alt.value(0.5),
).transform_filter(sel_year)


In [23]:
exercise_5 = (lines+points+text+mean_y).add_selection(sel_year)

In [ ]:
exercise_5

**QUESTION 6:** 

Create a chart that communicates the intersection of the selected 10 departments with the areas of research (publications). Restrict to 40 areas of research at most (the ones with more publications). Make it interactive in the following way: the two departments that are highlighted in the initial view must be highlighted here, but the rest of the information must be still visible.

First, we have to obtain the 40 areas of research with more articles published.

In order to do so first we create new dataset called 'data6' with only the needed columns for the question. 

In [25]:
data6 = data_proj2[["DEPT ACRONIMS","GRUPS ÀMBITS","ID_ARTICLE"]]

Then, group the areas of research and obtain the total number of articles published per area of research. 

When trying to do this step, we found out there are many departments that belong to one area of research. That means that the number of publications per area is repeated throughout the areas, because the same department (and number of publications) is in different areas.

The following table shows this phenomenon.

In [ ]:
data62 = data6.groupby(["GRUPS ÀMBITS"],sort=True)["DEPT ACRONIMS"].unique().to_frame()
data62

So, instead of only counting the publications of the areas of research, we also grouped by departments. This way we got the number of articles published by area and department.

In [27]:
data61 = data6.groupby(["GRUPS ÀMBITS","DEPT ACRONIMS"],sort=True)["ID_ARTICLE"].unique().to_frame().reset_index().rename(columns={"ID_ARTICLE":"NUM PAPERS"})
data61["NUM PAPERS"] = [len(data61["NUM PAPERS"][i]) for i in range(len(data61)) ]

The goal of these steps was to obtain the 40 areas of research with more articles published, so we filter the previously defined data set 'data62' with the areas that have at least 40 papers published. This filter reduced the number of areas to aproximately 40.

In [28]:
data63 = data61.loc[data61['NUM PAPERS']>40]

For the interactive part we define the multiple selection.

The next step is to plot the chart of the intersection between the 10 selected departments and the 40 areas of research. The departments can be selected interactively by clicking on the legend. 

Also, a panel with the name of the area of research will appear when the mouse is over the points. We though this would be useful because there are among 40 areas and it may be difficult to read which one belongs to which point.

In [29]:
# Condition of the cross selection
color = alt.condition(multi_sel, 
                      alt.Color('DEPT ACRONIMS:N',scale=alt.Scale(scheme='paired'), title = "Departments"),
                      alt.value('lightgray'))

# Scatterplot of intersection between areas and departments
plot6 = alt.Chart(data63).mark_circle().encode(
    alt.X('GRUPS ÀMBITS:N', axis=alt.Axis(labelFontSize=9, labelAngle=-45), title = "Fields groups"),
    alt.Y('DEPT ACRONIMS:N', axis=alt.Axis(labelFontSize=9), title = "Departments"),
    color = color,
    detail = 'GRUPS ÀMBITS:N',
    tooltip = ['GRUPS ÀMBITS']
).properties(
    title = 'Intersection between Departments and Areas of Research',
    width = 700, height = 150
).transform_filter(multi_sel).properties(width=800,height=400)

# Line plot of each department
plot6line = alt.Chart(data63).mark_line().encode(
    alt.X('GRUPS ÀMBITS:N'),
    alt.Y('DEPT ACRONIMS:N'),
    color = color
).properties(width=800,height=400)

# Combined plot
plot61 = plot6+plot6line
exercise_6 = plot61.interactive().add_selection(multi_sel)

In [ ]:
exercise_6

In [ ]:
part1 = alt.HConcatChart(hconcat=(exercise_5, exercise_1))
part2 = alt.HConcatChart(hconcat=(part1, exercise_6))
part3 = alt.HConcatChart(hconcat=(ex, plot42)).properties(resolve = alt.Resolve(scale=alt.LegendResolveMap(color=alt.ResolveMode('independent'))))
part4 = alt.VConcatChart(vconcat=(part2, part3))
part4